## Воспроизведение результатов работы - DoRA: Weight-Decomposed Low-Rank Adaptation 
- [Archive](https://arxiv.org/html/2402.09353v6) 
- [GitHub](https://github.com/NVlabs/DoRA)

In [ ]:
import os
import sys
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader

In [ ]:
MODEL_PATH = "google/gemma-3-1b-it"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
CACHE_DIR = "./cache_dir"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

torch.device(DEVICE)

print(
    DEVICE,
    torch.cuda.current_device()
)

cuda:0 0


#### Загрузка токенайзера и модели

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, cache_dir=CACHE_DIR)

tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [4]:
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, cache_dir=CACHE_DIR)

In [5]:
model.to(DEVICE)

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((11

In [6]:
print(torch.cuda.memory_allocated())
print(str(torch.cuda.memory_allocated() / 1024 ** 2) + " MB")
print(str(torch.cuda.memory_reserved() / 1024 ** 2) + " MB")

4025759744
3839.263671875 MB
3930.0 MB


### DoRA

.peft скопированна с репозитория - https://github.com/NVlabs/DoRA

In [7]:
sys.path.append(os.path.join(os.getcwd(), "peft/src/"))
from peft import (  # noqa: E402
    LoraConfig,
    PeftModel,
    DoraConfig,
    BottleneckConfig,
    PrefixTuningConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

In [ ]:
#Определяем конфигурацию для DoRa
dora_config = DoraConfig(
    r=16,
    lora_alpha=16,  # Масштабирующий коэффициент
    target_modules=["q_proj", "v_proj"],  # Модули для тонкой настройки (например, attention слои)
    lora_dropout=0.05,  # Дропаут
    bias="none",  # Без смещения
    task_type="CAUSAL_LM",  # Тип задачи
    dora_simple=True,  # Упрощённый режим DoRA
    Wdecompose_target_modules=["q_proj", "v_proj"]  # Модули для декомпозиции весов
)

In [ ]:
# Загрузка датасета
data_path = "./dataset/hellaswag"
cutoff_len = 32 # Максимальная длина последовательности
val_set_size = 2000  # Размер валидационной выборки

In [ ]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    result["labels"] = result["input_ids"].copy()
    return result

# Генерация промпта
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction: {data_point["instruction"]}
### Input: {data_point["input"]}
### Response: {data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: {data_point["instruction"]}
### Response: {data_point["output"]}"""

# Токенизация с генерацией промпта
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

#### Загрузка данных

In [ ]:
data = load_dataset(data_path)

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 39905
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 10042
    })
})

In [12]:
if val_set_size > 0:
    #Разделили на часть обучени и валидации
    train_val = data["train"].train_test_split(test_size=val_set_size, shuffle=True, seed=42)
    print(train_val)
    train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 37905
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 2000
    })
})


Map:   0%|          | 0/37905 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
model = prepare_model_for_int8_training(model, use_gradient_checkpointing=False)

In [13]:
def preprocess_function(examples, tokenizer):
    prompts = [generate_prompt(instruction) for instruction in examples["instruction"]]
    inputs = tokenizer(prompts, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    # Метки — это правильные ответы (например, ending1, ending2 и т.д.)
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": examples["answer"]}

In [14]:
def extract_answer(sentence):
    sentence = sentence.strip()
    pred_answers = re.findall(r'ending1|ending2|ending3|ending4', sentence)
    return pred_answers[0] if pred_answers else ""

#### Trainer для обучения модели

In [ ]:
batch_size = 16
micro_batch_size = 16
gradient_accumulation_steps = batch_size // micro_batch_size
num_epochs = 3
learning_rate = 2e-4
eval_step = 2000000 #Данное значение взято специально, с расчетом не производить вычисление метрики приобучении
save_step = 2000000
output_dir = "./dora-ngemma" #Директория для сохранения результата - конфига и самого адаптера

# Инициализация Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=TrainingArguments(
        per_device_train_batch_size=micro_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.0,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        eval_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_step if val_set_size > 0 else None,
        save_steps=save_step,
        output_dir=output_dir,
        save_total_limit=3,
        load_best_model_at_end=True if val_set_size > 0 else False
    ),
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

# Отключение кэша для обучения
model.config.use_cache = False

#### Обучение завершилось, но при попытке оценки модели, во время обучения, возникали ошибки. Во время обучения было принято решение не оценивать качество модели (не вычислять метрику), а оценить после. Оценка полученного результата представлена ниже

In [ ]:
# Обучение
trainer.train()

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss,Validation Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.model.embed_tokens.weight', 'base_model.model.lm_head.0.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [19]:
# Сохранение модели
model.save_pretrained(output_dir)

### Evaluate
Оценка производилась на тестовой части бенчмарка HellaSwag

In [15]:
output_dir = "./dora-ngemma"

#### Загрузка адаптера для модели

In [ ]:
model = PeftModel.from_pretrained(model, output_dir) #Загрузка адаптера

base_model.model.model.layers.0.self_attn.q_proj.weight_m_wdecomp.weight is trainable
base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight is trainable
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight is trainable
base_model.model.model.layers.0.self_attn.v_proj.weight_m_wdecomp.weight is trainable
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight is trainable
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight is trainable
base_model.model.model.layers.1.self_attn.q_proj.weight_m_wdecomp.weight is trainable
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight is trainable
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight is trainable
base_model.model.model.layers.1.self_attn.v_proj.weight_m_wdecomp.weight is trainable
base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight is trainable
base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight is trainable
base_model.model.model.layers.2.self_attn.q_proj

In [18]:
model.to(DEVICE)

PeftModelForCausalLM(
  (base_model): DoraModel(
    (model): PeftModelForCausalLM(
      (base_model): DoraModel(
        (model): Gemma3ForCausalLM(
          (model): Gemma3TextModel(
            (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
            (layers): ModuleList(
              (0-25): 26 x Gemma3DecoderLayer(
                (self_attn): Gemma3Attention(
                  (q_proj): Linear(
                    in_features=1152, out_features=1024, bias=False
                    (lora_dropout): Dropout(p=0.05, inplace=False)
                    (weight_m_wdecomp): Linear(in_features=1, out_features=1024, bias=False)
                    (lora_A): Linear(in_features=1152, out_features=16, bias=False)
                    (lora_B): Linear(in_features=16, out_features=1024, bias=False)
                  )
                  (k_proj): Linear(in_features=1152, out_features=256, bias=False)
                  (v_proj): Linear(
                    in_featu

#### Загрузка тестовой части бенчмарка и преобразование (формирование промпта и токенизация)

In [20]:
test_data = data["test"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

#### Функция для вычисления accuracy

In [ ]:
def compute_accuracy(model, dataset, tokenizer, device):
    dataloader = DataLoader(dataset, batch_size=2)

    correct = 0
    total = 0

    cnt = 0

    with torch.no_grad():
        for batch in dataloader:
            instructions = batch['instruction']
            labels = batch['answer']

            tokenized_inputs = tokenizer(instructions, return_tensors='pt', padding=True, truncation=True)
            tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

            outputs = model(input_ids=tokenized_inputs['input_ids'],
                            attention_mask=tokenized_inputs['attention_mask'])

            predicted_token_ids = outputs.logits.argmax(dim=-1)

            predictions = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
            predictions = [extract_answer(pred.split("### Response:")[-1].strip()) for pred in predictions]

            correct += sum(pred == label for pred, label in zip(predictions, labels))
            total += len(labels)
            cnt += 1

            if cnt % 200 == 0:
                print(f"num batch: {cnt*2}, accuracy: {correct / total}")

    accuracy = correct / total
    return accuracy

In [22]:
accuracy_score = compute_accuracy(model, test_data, tokenizer, DEVICE)
print(f"Accuracy: {accuracy_score}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


num batch: 400, accuracy: 0.235
num batch: 800, accuracy: 0.24875
num batch: 1200, accuracy: 0.24916666666666668
num batch: 1600, accuracy: 0.251875
num batch: 2000, accuracy: 0.2565
num batch: 2400, accuracy: 0.2525
num batch: 2800, accuracy: 0.24857142857142858
num batch: 3200, accuracy: 0.253125
num batch: 3600, accuracy: 0.2577777777777778
num batch: 4000, accuracy: 0.2565
num batch: 4400, accuracy: 0.25272727272727274
num batch: 4800, accuracy: 0.24916666666666668
num batch: 5200, accuracy: 0.24903846153846154
num batch: 5600, accuracy: 0.24660714285714286
num batch: 6000, accuracy: 0.24566666666666667
num batch: 6400, accuracy: 0.24671875
num batch: 6800, accuracy: 0.24661764705882352
num batch: 7200, accuracy: 0.24847222222222223
num batch: 7600, accuracy: 0.2488157894736842
num batch: 8000, accuracy: 0.246375
num batch: 8400, accuracy: 0.24654761904761904
num batch: 8800, accuracy: 0.24806818181818183
num batch: 9200, accuracy: 0.2501086956521739
num batch: 9600, accuracy: 0.25

### Выводы

- Можно заметить очень низкое значение метрики модели (0.25)
- Размер рассмотренной модели существенно меньше, моделей в работе (Llama 7B, 8B, 13B, против Gemma 1B). Выбор модели основывался чисто на требованиях к ресурсам, на какой модели можно было быстро проверить работоспособность
- Для настройки были выбраны следующие параметры: r = 16, lr = 2e-4, количество эпох = 3
- Размер обучающих данных очень маленький (около 50 мб). В исследовании подчеркивается, что создавался общий датасет (обучение производилось на всех рассмотренных бенчмарках сразу)
- Во время обучения не производилась оценка модели, что позволило бы остановить и скорректировать процесс обучения (нужно доработать)
- В результате работы был получен baseline, который можно применить в дальнейшем для тонкой настройки и оценки модели. Не получилось получить результатов, по метрикам, сопостовимых с результатами работы (https://github.com/NVlabs/DoRA), но дальнейшие эксперименты, возможно, приблизят к ним.